# Processamento de Dados
Este notebook pertence ao projeto: Case Téc­ni­co Data Sci­en­ce - iFo­od acessível em \
https://github.com/paulolaque/ifood-case.git

## Procedimentos iniciais

### Importação de bibliotecas

In [30]:
import os
import tarfile
import urllib.request
import sys



# Este notebook executa em pyspark, favor instalar caso necessário
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
    col,
    when,
    lit,
    explode,
    collect_set,
    collect_list,
    array,
    array_contains,
    sum as spark_sum,
    isnan,
    to_date,
    datediff,
    max as spark_max,
    year,
    month,
    weekofyear,
    struct
)
except ImportError:
    print("PySpark não está instalado. Instale com: pip install pyspark")
    sys.exit(1)

### Configuração de diretórios

In [31]:
# Detectar se está rodando no Databricks, pois o padrão de diretórios podem ser diferentes de uma máquina local
running_in_databricks = os.path.exists("/databricks/driver")

# Ajusta o caminho raiz do projeto
if running_in_databricks:
    project_root = "/dbfs" + os.getcwd().split("/notebooks")[0]  # raiz do workspace em DBFSpara Databricks
else:
    project_root = os.path.abspath("..")  # pasta acima de notebooks/

# Cria string de diretório
raw_data_path = os.path.join(project_root, "data", "raw")
tar_gz_path = os.path.join(raw_data_path, "ds-technical-evaluation-data.tar.gz")

# Cria a pasta se não existir
os.makedirs(raw_data_path, exist_ok=True)

### Configuração de sessão Spark 

In [32]:
# Cria SparkSession se necessário
spark = SparkSession.builder \
    .appName("Load JSON Files") \
    .getOrCreate()

## Carregamento e Leitura dos Dados

### Download e extração de arquivos

In [33]:

# URL de download
url = "https://data-architect-test-source.s3.sa-east-1.amazonaws.com/ds-technical-evaluation-data.tar.gz"

# Baixa o arquivo se não existir
if not os.path.exists(tar_gz_path):
    print("Baixando arquivo...")
    urllib.request.urlretrieve(url, tar_gz_path)
    print("Download completo.")
else:
    print("Arquivo já existe.")

# Extrai os arquivos
print("Extraindo arquivos...")
with tarfile.open(tar_gz_path, "r:gz") as tar:
    tar.extractall(path=raw_data_path)
print("Extração completa.")

# Lista arquivos
extracted_path = os.path.join(raw_data_path, "ds-technical-evaluation-data")
print("Arquivos extraídos:")
for root, dirs, files in os.walk(extracted_path):
    for file in files:
        print(os.path.join(root, file))

Arquivo já existe.
Extraindo arquivos...
Extração completa.
Arquivos extraídos:
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._transactions.json
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._offers.json
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/profile.json
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/offers.json
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._profile.json
/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/transactions.json


###  Conversão para DataFrames do Spark

In [34]:
for file in os.listdir(extracted_path):
    print(file)


# Função para extrair em tabelas
def read_extracted_df(table_name: str):
    df = None  # Inicializa a variável df
    
    for file in os.listdir(extracted_path):
        # Verifica se o arquivo é o '.json'
        if table_name in file.lower() and file.endswith(".json"):
            # Detecta se precisa usar prefixo "file:/"
            if running_in_databricks:
                spark_path = "file:" + os.path.join(extracted_path, file)
            else:
                spark_path = os.path.join(extracted_path, file)

            print(f"Lendo {spark_path}")

            try:
                # Lê o arquivo JSON
                df = spark.read.json(spark_path)
                df = df.withColumn("source_file", lit(file))  # Adiciona a coluna do nome do arquivo
                break  # Para ao encontrar o arquivo ".json"
            except Exception as e:
                print(f"Erro ao ler {file}: {e}")
                
    return df  # Retorna o DataFrame


._transactions.json
._offers.json
profile.json
offers.json
._profile.json
transactions.json


In [35]:
df_profile = read_extracted_df('profile')
df_transactions=read_extracted_df('transactions')
df_offers=read_extracted_df('offers')

Lendo /workspaces/ifood-case/data/raw/ds-technical-evaluation-data/profile.json
Lendo /workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._transactions.json


25/04/17 17:08:26 WARN DataSource: All paths were ignored:
  file:/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._transactions.json


Erro ao ler ._transactions.json: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for JSON. It must be specified manually.
Lendo /workspaces/ifood-case/data/raw/ds-technical-evaluation-data/transactions.json


25/04/17 17:08:28 WARN DataSource: All paths were ignored:                      
  file:/workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._offers.json


Lendo /workspaces/ifood-case/data/raw/ds-technical-evaluation-data/._offers.json
Erro ao ler ._offers.json: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for JSON. It must be specified manually.
Lendo /workspaces/ifood-case/data/raw/ds-technical-evaluation-data/offers.json


## Transformação de Tabelas

### Desaninhamento de JSON

In [36]:


print('Antes:', df_transactions.count())

# Acesse diretamente os campos da coluna STRUCT 'value'
df_transactions = df_transactions.select("*", 
    col("value.amount").alias("amount"),
    col("value.offer_id").alias("offer_id"),
    col("value.offer id").alias("offer id"),
    col("value.reward").alias("reward")
)

print(f'Depois de desaninhar Json da coluna value: {df_transactions.count()}')

# Retirar coluna desaninhada
df_transactions=df_transactions.drop("value")

Antes: 306534
Depois de desaninhar Json da coluna value: 306534


Podemos ver que desaninhar não alterou o número de linhas

### Conversão de arrays em colunas

In [37]:


#  Explode os valores da lista para identificar os valores únicos
unique_channels = df_offers.select(explode(col("channels")).alias("channel")) \
                                  .distinct() \
                                  .rdd.flatMap(lambda row: row) \
                                  .collect()

#  Para cada valor possível, cria uma nova coluna binária (1 se presente, 0 se não)
for ch in unique_channels:
    df_offers = df_offers.withColumn(
        f"channel_{ch}",
        when(array_contains(col("channels"), ch), lit(1)).otherwise(lit(0))
    )
df_offers=df_offers.drop("channels")

## Manipulação e Limpeza dos Dados

### Remover linhas duplicadas

In [38]:
def verificar_duplicatas(df, nome_df="DataFrame"):
    total = df.count()
    distintos = df.distinct().count()
    duplicadas = total - distintos
    print(f"\n {nome_df}:")
    print(f" - Total de linhas: {total}")
    print(f" - Linhas distintas: {distintos}")
    print(f" - Linhas duplicadas: {duplicadas}")

verificar_duplicatas(df_transactions, "df_transactions")
verificar_duplicatas(df_offers, "df_offers")
verificar_duplicatas(df_profile, "df_profile")


 df_transactions:
 - Total de linhas: 306534
 - Linhas distintas: 306137
 - Linhas duplicadas: 397

 df_offers:
 - Total de linhas: 10
 - Linhas distintas: 10
 - Linhas duplicadas: 0

 df_profile:
 - Total de linhas: 17000
 - Linhas distintas: 17000
 - Linhas duplicadas: 0


Como temos uma série histórica de eventos sem id por evento (a tabela transactions não tem id) e temos apenas o tempo (time_since_test_start), o tipo de evento e o id da conta como referência, não conseguimos identificar se um evento de fato esta duplicado na tabela. Porem, como verificamos que apenas 0,13% da base esta duplicada, acredito que não será significante na nossa base. Por isso vamos excluir.

In [39]:
df_transactions = df_transactions.dropDuplicates()
df_offers = df_offers.dropDuplicates()
df_profile = df_profile.dropDuplicates()

verificar_duplicatas(df_transactions, "df_transactions")
verificar_duplicatas(df_offers, "df_offers")
verificar_duplicatas(df_profile, "df_profile")



 df_transactions:
 - Total de linhas: 306137
 - Linhas distintas: 306137
 - Linhas duplicadas: 0

 df_offers:
 - Total de linhas: 10
 - Linhas distintas: 10
 - Linhas duplicadas: 0

 df_profile:
 - Total de linhas: 17000
 - Linhas distintas: 17000
 - Linhas duplicadas: 0


### Identificar colunas duplicadas

In [40]:
display(df_transactions.describe())

DataFrame[summary: string, account_id: string, event: string, time_since_test_start: string, source_file: string, amount: string, offer_id: string, offer id: string, reward: string]

In [41]:
# Como offer id e offer_id aparentemente são colunas que se complementam vamos testar se isso é verdade: quando ambas são não nulas se não temos divergência de valores

# Filtra linhas em que ambas as colunas são não nulas e diferentes
df_diff = df_transactions.filter(
    col("offer id").isNotNull() & 
    col("offer_id").isNotNull() & 
    (col("offer id") != col("offer_id"))
)

# Conta essas linhas
diff_count = df_diff.count()
print(f"Número de linhas com 'offer id' diferente de 'offer_id': {diff_count}")


Número de linhas com 'offer id' diferente de 'offer_id': 0


In [42]:
display(df_profile.describe())

DataFrame[summary: string, age: string, credit_card_limit: string, gender: string, id: string, registered_on: string, source_file: string]

In [43]:
display(df_offers.describe())

DataFrame[summary: string, discount_value: string, duration: string, id: string, min_value: string, offer_type: string, source_file: string, channel_mobile: string, channel_email: string, channel_social: string, channel_web: string]

Apenas olhandos o mínimo e máximo de cada coluna podemos ver que não temos possíveis colunas duplicadas.
 Se fossem iguais poderiamos testar se realmente são iguais,mas não precisamos.

### Tratamento de valores nulos 

#### Verificação de valores nulos

In [44]:


def check_nulls(df, df_name="DataFrame"):
    print(f"\n>>> Verificando nulos em: {df_name}")
    nulls_df = df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
    ])
    nulls_df.show()

dfs = {
    "df_profile": df_profile,
    "df_transactions": df_transactions,
    "df_offers": df_offers
}

for name, df in dfs.items():
    check_nulls(df, name)



>>> Verificando nulos em: df_profile
+---+-----------------+------+---+-------------+-----------+
|age|credit_card_limit|gender| id|registered_on|source_file|
+---+-----------------+------+---+-------------+-----------+
|  0|             2175|  2175|  0|            0|          0|
+---+-----------------+------+---+-------------+-----------+


>>> Verificando nulos em: df_transactions


+----------+-----+---------------------+-----------+------+--------+--------+------+
|account_id|event|time_since_test_start|source_file|amount|offer_id|offer id|reward|
+----------+-----+---------------------+-----------+------+--------+--------+------+
|         0|    0|                    0|          0|167184|  272955|  172135|272955|
+----------+-----+---------------------+-----------+------+--------+--------+------+


>>> Verificando nulos em: df_offers
+--------------+--------+---+---------+----------+-----------+--------------+-------------+--------------+-----------+
|discount_value|duration| id|min_value|offer_type|source_file|channel_mobile|channel_email|channel_social|channel_web|
+--------------+--------+---+---------+----------+-----------+--------------+-------------+--------------+-----------+
|             0|       0|  0|        0|         0|          0|             0|            0|             0|          0|
+--------------+--------+---+---------+----------+-----------

#### Imputação de valores nulos em numéricos

In [45]:


# Verificando a hipótese de que os valores nulos são na verdade zero nas colunas numéricas
print("Tipos das colunas:")
print("df_profile:")
df_profile.select("credit_card_limit").printSchema()

print("df_transactions:")
df_transactions.select("amount", "reward").printSchema()

# Contando valores zero
zero_credit_card_limit = df_profile.filter(col("credit_card_limit") == 0).count()
print(f"\nNúmero de registros com credit_card_limit = 0: {zero_credit_card_limit}")

zero_amount = df_transactions.filter(col("amount") == 0).count()
print(f"Número de registros com amount = 0: {zero_amount}")

zero_reward = df_transactions.filter(col("reward") == 0).count()
print(f"Número de registros com reward = 0: {zero_reward}")


Tipos das colunas:
df_profile:
root
 |-- credit_card_limit: double (nullable = true)

df_transactions:
root
 |-- amount: double (nullable = true)
 |-- reward: double (nullable = true)


Número de registros com credit_card_limit = 0: 0


Número de registros com amount = 0: 0


Número de registros com reward = 0: 0


In [46]:
# Como não temos valor zero para as três colunas numéricas com dados faltantes (reward,amount e credit_card_limit), faz sentido supor que quando são nulas são zero, logo vamos inputar como zero esses dados. Para não perder informação vamos criar colunas informando quando elas foram nulas.


# Para a tabela df_profile
df_profile = df_profile.withColumn(
    "credit_card_limit_is_null", 
    when(col("credit_card_limit").isNull(), 1).otherwise(0)
).fillna({"credit_card_limit": 0})  # Preenche os nulos de credit_card_limit com 0

# Para a tabela df_transactions
df_transactions = df_transactions.withColumn(
    "amount_is_null", 
    when(col("amount").isNull(), 1).otherwise(0)
).withColumn(
    "reward_is_null", 
    when(col("reward").isNull(), 1).otherwise(0)
).fillna({"amount": 0, "reward": 0})  # Preenche os nulos de amount e reward com 0


#### Imputação de valores nulos com coluna auxiliar

In [47]:


# Como vimos que 'offer id' e 'offer_id' se complementam e não contem divergência de dados vamos unificá-las
# Atualiza 'offer_id' com valor de 'offer id' se for null
df_transactions = df_transactions.withColumn(
    "offer_id",
    when(col("offer_id").isNull(), col("offer id")).otherwise(col("offer_id"))
)

# Remove a coluna com dados duplicados
df_transactions = df_transactions.drop("offer id")

#### Imputação de valores nulos categóricos

In [48]:
df_profile=df_profile.fillna({
    "gender": "nao_preenchido"
})


### Remoção de colunas irrelevantes

In [49]:
#vamos também remover as colunas source que não adicionam valor a nossa análise por ter sempre o mesmo valor e dizer apenas o nome da tabela. Isso vai melhorar a organização da tabela para não ter colunas com nomes duplicados
df_transactions=df_transactions.drop("source_file")
df_offers=df_offers.drop("source_file")
df_profile=df_profile.drop("source_file")

### Remoção de Outliers

In [50]:


def remover_outliers(df, nome_df="df"):
    print(f"\nRemovendo outliers em: {nome_df}")
    numeric_cols = [f.name for f in df.schema.fields if str(f.dataType) in ('IntegerType', 'DoubleType', 'LongType', 'FloatType')]
    original_count = df.count()
    df_filtrado = df

    for col_name in numeric_cols:
        quantiles = df_filtrado.approxQuantile(col_name, [0.25, 0.75], 0.01)
        if len(quantiles) < 2:
            continue

        q1, q3 = quantiles
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr

        df_filtrado = df_filtrado.filter((col(col_name) >= lower) & (col(col_name) <= upper))

    final_count = df_filtrado.count()
    total_removed = original_count - final_count

    print(f"{total_removed} linhas removidas por outliers em {nome_df}")
    print(f"Total final de linhas: {final_count}")
    return df_filtrado

# Aplicando a função nos três datasets
df_transactions = remover_outliers(df_transactions, "df_transactions")
df_offers = remover_outliers(df_offers, "df_offers")
df_profile = remover_outliers(df_profile, "df_profile")


Removendo outliers em: df_transactions


0 linhas removidas por outliers em df_transactions
Total final de linhas: 306137

Removendo outliers em: df_offers
0 linhas removidas por outliers em df_offers
Total final de linhas: 10

Removendo outliers em: df_profile
0 linhas removidas por outliers em df_profile
Total final de linhas: 17000


## Preparação do Dataset Unificado

#### Junção de múltiplas tabelas (joins)

In [51]:
# Faz o LEFT JOIN entre transações e perfis e remover colunas de ids duplicados.
print('Total de linhas Antes:',df_transactions.count())
df_joined = df_transactions.join(df_profile, df_transactions.account_id == df_profile.id, how="left")
df_joined=df_joined.drop("id")
df_joined = df_joined.join(df_offers, df_joined.offer_id == df_offers.id, how="left")
df_joined=df_joined.drop("id")
print('Total de linhas depois:',df_joined.count())

Total de linhas Antes: 306137


Total de linhas depois: 306137


#### Checagem de integridade após merge

In [52]:


def contar_nulos(df, nome_df="DataFrame"):
    print(f"\nValores nulos no {nome_df}:\n")
    display(df.select([
        spark_sum(
            when(col(c).isNull(), 1).otherwise(0)
        ).alias(c) for c in df.columns
    ]))

# Exemplo para df_joined
contar_nulos(df_joined, "df_joined")


Valores nulos no df_joined:



DataFrame[account_id: bigint, event: bigint, time_since_test_start: bigint, amount: bigint, offer_id: bigint, reward: bigint, amount_is_null: bigint, reward_is_null: bigint, age: bigint, credit_card_limit: bigint, gender: bigint, registered_on: bigint, credit_card_limit_is_null: bigint, discount_value: bigint, duration: bigint, min_value: bigint, offer_type: bigint, channel_mobile: bigint, channel_email: bigint, channel_social: bigint, channel_web: bigint]

#### Ajuste de Nulos Pós Merge

In [53]:
# Como na tabela transactions haviam offer_id nulos vamos prencher esses valores com zero, ou seja não houve oferta e vamos criar uma coluna para identificar quando não houve oferta.


#  Criar coluna binária indicando se offer_id é nulo
df_joined = df_joined.withColumn("is_offer_id_null", when(col("offer_id").isNull(), 1).otherwise(0))

#  Lista de colunas para preencher nulos com zero
colunas_para_preencher = [
    "offer_id", "discount_value", "duration", "min_value",
    "channel_mobile", "channel_email", "channel_social", "channel_web"
]

#  Substituir nulos por 0 nessas colunas
for coluna in colunas_para_preencher:
    df_joined = df_joined.withColumn(coluna, when(col(coluna).isNull(), 0).otherwise(col(coluna)))

## Engenharia de Variáveis

### Transformar Data em tempo

In [54]:


# Vamos criar a coluna de data para tempo para o modelo conseguir interpretar 

#  Converter 'registered_on' para data no formato yyyyMMdd
df_joined = df_joined.withColumn("registered_on_date", to_date(col("registered_on").cast("string"), "yyyyMMdd"))

#  Obter a data mais recente da coluna 'registered_on_date'
max_data = df_joined.agg(spark_max("registered_on_date").alias("max_date")).collect()[0]["max_date"]

# Criar nova coluna com diferença em dias até a data mais recente
df_joined = df_joined.withColumn("dias_desde_registro", datediff(lit(max_data), col("registered_on_date")))

### Extração de features

#### Criação de variáveis temporais derivadas

In [55]:


# Criar colunas de ano, mês e semana
df_joined = df_joined \
    .withColumn("ano_registro", year(col("registered_on_date"))) \
    .withColumn("mes_registro", month(col("registered_on_date"))) \
    .withColumn("semana_registro", weekofyear(col("registered_on_date")))

# Remover coluna antiga
df_joined = df_joined.drop("registered_on")

## Reformatação da Tabela

In [56]:

#  Filtrar ofertas recebidas
offers_received = df_joined.filter(col("event") == "offer received")

#  Adicionar janela de duração da oferta
offers_received = offers_received.withColumn("offer_window_end", col("time_since_test_start") + col("duration"))

#  Transações
transactions = df_joined.filter(col("event") == "transaction") \
    .select("account_id", "time_since_test_start") \
    .withColumnRenamed("time_since_test_start", "transaction_time")

#  Verificar se houve transação no intervalo da oferta
joined = offers_received.join(
    transactions,
    on="account_id",
    how="left"
).filter(
    (col("transaction_time") >= col("time_since_test_start")) &
    (col("transaction_time") <= col("offer_window_end"))
)

#  Marcar conversões
converted = joined.groupBy("account_id", "offer_id", "time_since_test_start") \
    .agg(spark_max(lit(1)).alias("converted"))

#  Preencher 0 para quem não converteu
offers = offers_received.join(
    converted,
    on=["account_id", "offer_id", "time_since_test_start"],
    how="left"
).withColumn("converted", when(col("converted").isNull(), 0).otherwise(1))

## Salvamento

### Salvar em Parquet

In [57]:
    # Caminho para salvar os dados processados
    processed_data_path = os.path.join(project_root, "data", "processed")

    # Cria a pasta se não existir
    os.makedirs(processed_data_path, exist_ok=True)

    # Caminho completo do arquivo Parquet
    parquet_output_path_transformed = os.path.join(processed_data_path, "unificado_transformed.parquet")
    parquet_output_path = os.path.join(processed_data_path, "unificado.parquet")

    # Salva como Parquet
    print(f"Salvando dados em: {parquet_output_path_transformed}")
    offers.write.mode("overwrite").parquet(parquet_output_path_transformed)
    print(f"Salvando dados em: {parquet_output_path}")
    df_joined.write.mode("overwrite").parquet(parquet_output_path)
    print("Dados salvos com sucesso.")


Salvando dados em: /workspaces/ifood-case/data/processed/unificado_transformed.parquet


Salvando dados em: /workspaces/ifood-case/data/processed/unificado.parquet


Dados salvos com sucesso.
